In [2]:
import pandas as pd
import pyodbc

In [3]:
cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
            "Server=DWHDB;"
            "Database=DWBIBFI2_ODS;"
            "UID=Data_Inovasion;"
            "PWD=Inovasion999;")
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()

In [4]:
query = """
WITH daily_aging AS(
 SELECT
        CAST(AgingDate AS date) AS AgingDate,
        TRIM(AgreementNo) AS AgreementNo,
        DefaultStatus,
        DaysOverdue
    FROM DWBIBFI2_ODS.dbo.ODS_DailyAging WITH (NOLOCK)
    WHERE AgingDate BETWEEN DATEADD(DAY, -1,'2023-07-01') AND '2023-07-31'
    UNION
    SELECT
        CAST(AgingDate AS date) AS AgingDate,
        TRIM(AgreementNo) AS AgreementNo,
        DefaultStatus,
        DaysOverdue
    FROM DWBIBFI2_ODS.dbo.ODS_DailyAging_Archieve_Daily WITH (NOLOCK)
    WHERE AgingDate BETWEEN DATEADD(DAY, -1,'2023-07-01') AND '2023-07-31'
),
ral AS (
    SELECT
        [CGName],
        [BranchName],
        [CustomerName],
        [AgreementNo],
        [AssetTypeID],
        [ProductType],
        [DefaultStatus],
        [DaysOverDueTolerance],
        [BucketGroup],
        [Requestdate],
        [PrintedDate],
        [RALExpectedEXPDate],
        [RALExpiredDate],
        [RALonRequestReason],
        [RALReleaseReason],
        [ApprovalNote],
        [ReleaseNotes],
        [ExecutorID],
        [ExecutorName],
        [Jabatan] as  [JABATAN COLLECTOR],
        [CollectorID],
        [Collectorname],
        [RALNO],
        [Result],
        [CaseCategory],
        [RepossesDate],
        [CollectorFrom],
        [SK_Time],
        [RAL_Date],
        [Upd_Date],
        CONVERT(varchar, (DATEADD(DAY, 1, EOMONTH(PrintedDate, -1))), 112) AS SK_Time_New,
        ROW_NUMBER() OVER(PARTITION BY RALNO ORDER BY RALNO) AS ral_dedup
    FROM
    [DW-DMCOLL].Collection_Dev.dbo.RAL_Monitoring WITH (NOLOCK)
    WHERE
        CONVERT(varchar, (DATEADD(DAY, 1, EOMONTH(PrintedDate, -1))), 112) = '20230701' --SK_Time_New
        AND PrintedDate IS NOT NULL
),
ral_printed_rnk AS(
    SELECT
        *,
--        PATINDEX('%[a-z]%', ExecutorID) AS letterIdx,
--        PATINDEX('%[0-9]%', ExecutorID) AS digitIdx,
--        CASE
--            WHEN ExecutorName = 'PT SOLUSI PRIMA UTAMA' AND AgreementNo = '4242106979' THEN 'RICKY HARSANDI'
--            ELSE ExecutorName
--        END AS [EXECUTOR NAME],
        CASE
            WHEN ProductType IN ('NDF Car', 'DF Used', 'DF Used', 'NCF') THEN 'CAR'
            ELSE ProductType
        END AS [PRODUCT GROUP],
        CASE
            WHEN DefaultStatus IN ('NA', 'NPL') THEN 'NPL'
            ELSE DefaultStatus
        END AS [DEFAULT STATUS],
        CASE
            WHEN PATINDEX('%[0-9]%', ExecutorID) > PATINDEX('%[a-z]%', ExecutorID) AND PATINDEX('%[a-z]%', ExecutorID) > 0
                AND LEN(ExecutorID) - PATINDEX('%[0-9]%', ExecutorID) BETWEEN 2 AND 4
                THEN 'Internal'
 --               WHEN ExecutorName = 'PT SOLUSI PRIMA UTAMA' and AgreementNo = '4242106979' THEN 'Mitra'
                WHEN LEFT(ExecutorName,2) = 'PT' OR RIGHT(ExecutorName,2) = 'PT' or RIGHT(ExecutorName,3) = 'PT.' THEN 'PT'
                ELSE 'Mitra'
        END as [JENIS EXECUTOR],
        DENSE_RANK() OVER(PARTITION BY AgreementNo ORDER BY PrintedDate DESC) AS rnk
    FROM ral
    WHERE ral_dedup = 1
),
ral_date AS(
    SELECT
    t1.*,
    CASE
     WHEN t1.rnk = 1 THEN pel.TanggalPelunasan
     ELSE NULL
    END AS TanggalPelunasan,
    CASE
     WHEN t1.rnk = 1 THEN pen.RepossesDate
     ELSE NULL
    END AS TanggalPenarikan,
    CASE
     WHEN t1.rnk = 1 THEN pen.InventoryDate
     ELSE NULL
    END AS TanggalInventori
    FROM
    ral_printed_rnk t1
    left join [DW-DMCOLL].Collection_Dev.dbo.Coll_IncentivePelunasan_Daily pel WITH (NOLOCK)
    ON TRIM(t1.AgreementNo) = TRIM(pel.AgreementNo) AND pel.SK_Time = CONVERT(VARCHAR(8), DATEADD(day, -1, CAST(GETDATE() as DATE)), 112) -- H-1 for curr month, EOM for historical
    left join [DW-DMCOLL].Collection_Dev.dbo.Coll_IncentivePenarikan_Daily pen WITH (NOLOCK)
    ON TRIM(t1.AgreementNo) = TRIM(pen.AgreementNo) AND pen.SK_Time = CONVERT(VARCHAR(8), DATEADD(day, -1, CAST(GETDATE() as DATE)), 112) -- H-1 for curr month, EOM for historical
),
ral_begin_end AS(
    SELECT
    *,
    DATEADD(day, -1, CAST(PrintedDate as DATE)) AS BeginDate,
    CASE
        WHEN TanggalPelunasan IS NOT NULL THEN TanggalPelunasan
        WHEN TanggalInventori IS NOT NULL THEN TanggalInventori
        WHEN TanggalPenarikan IS NOT NULL THEN TanggalPenarikan
        WHEN RALExpiredDate >= CAST(GETDATE() AS DATE) THEN DATEADD(day, -1, CAST(GETDATE() as DATE)) -- curr month
    ELSE CAST(RALExpiredDate AS DATE)
    END EndDate
    FROM ral_date
),
ral_pastduedays AS(
SELECT
 t1.*,
    t2.DaysOverdue AS BeginPastDueDays,
    t2.DefaultStatus AS BeginDefaultStatus,
    t3.DaysOverdue AS EndPastDueDays,
    t3.DefaultStatus AS EndDefaultStatus,
    CASE
        WHEN t2.DaysOverdue = 0 THEN 0
        ELSE FLOOR((t2.DaysOverdue -1)/30) * 30 +1
    END AS BeginBucket,
    CASE
        WHEN t3.DaysOverdue = 0 THEN 0
        ELSE FLOOR((t3.DaysOverdue -1)/30) * 30 +1
    END AS EndBucket
FROM
ral_begin_end t1
left join daily_aging t2 ON TRIM(t1.AgreementNo) = t2.AgreementNo and CAST(t1.BeginDate AS date) = t2.AgingDate
left join daily_aging t3 ON TRIM(t1.AgreementNo) = t3.AgreementNo and CAST(t1.EndDate AS date) = t3.AgingDate
),
ral_colleffectiveness AS (
SELECT
t1.*,
CASE
    WHEN t1.TanggalPelunasan IS NOT NULL THEN 'Pelunasan'
    WHEN t1.TanggalInventori IS NOT NULL THEN 'Inventory'
    WHEN t1.TanggalPenarikan IS NOT NULL THEN 'Penarikan'
    WHEN t1.TanggalPelunasan IS NULL
        and t1.TanggalInventori IS NULL
        and t1.TanggalPenarikan IS NULL
        and t1.EndPastDueDays IS NULL
        THEN 'LUNAS'
    WHEN t1.BeginPastDueDays <> 0 and t1.EndPastDueDays = 0 THEN 'Back to Current'
    WHEN t1.BeginPastDueDays = 0 and t1.EndPastDueDays > 0 THEN 'Roll Rate'
    WHEN t1.BeginBucket = t1.EndBucket THEN 'Same Aging'
    WHEN t1.BeginBucket < t1.EndBucket THEN CONCAT('Roll to ', EndBucket)
    WHEN t1.BeginPastDueDays > t1.EndPastDueDays THEN 'Roll Back'
    ELSE 'N/A'
END CollEffectiveness
FROM ral_pastduedays t1
),
ral_dna AS(
SELECT
    t1.*,
    dnaPrevMonth.[OSP Real Time] as [OSP Bulan Sebelumnya],
    dnaCurrMonth.[OSP Real Time] as [OSP Bulan Berjalan],
    dnaPrevMonth.WilayahTagihID as WilayahTagihID_dna,
    dnaPrevMonth.WilayahTagihName as WilayahTagihName_dna,
    dnaPrevMonth.SubCGID as SubCGID_dna,
    dnaPrevMonth.CGID as CGID_dna,
    dnaPrevMonth.CGName as CGName_dna,
    dnaPrevMonth.[Hari Keterlambatan] as DaysOverDue_dna,
    dnaPrevMonth.[Default Status] as DefaultStatus_dna,
    CASE
        WHEN t1.CollEffectiveness IN ('Pelunasan', 'Inventory', 'Penarikan', 'LUNAS', 'Back to Current') AND t1.rnk=1
            THEN 'Sukses'
        WHEN t1.CollEffectiveness = 'Roll Back' THEN 'Sukses Parsial'
        ELSE 'Gagal'
    END [RESULT GROUP],
    CASE
        WHEN t1.CollEffectiveness IN ('Pelunasan', 'Inventory', 'Penarikan', 'LUNAS', 'Back to Current', 'Roll Back') AND t1.rnk=1
            THEN 'Sukses'
        ELSE 'Gagal'
    END [RESULT GROUP 2]
FROM ral_colleffectiveness t1
left join [DW-DMCOLL].Collection_Dev.dbo.DNA_Collection dnaPrevMonth WITH (NOLOCK) on trim(t1.AgreementNo) = trim(dnaPrevMonth.[Nomor Kontrak]) and dnaPrevMonth.AgingDate = '2023-06-30'
left join [DW-DMCOLL].Collection_Dev.dbo.DNA_Collection dnaCurrMonth WITH (NOLOCK) on trim(t1.AgreementNo) = trim(dnaCurrMonth.[Nomor Kontrak]) and dnaCurrMonth.AgingDate =  DATEADD(day, -1, CAST(GETDATE() as DATE)) -- H-1 for curr month, EOM for historical
),
ral_dna_bucket AS(
SELECT
*,
CASE
    WHEN DaysOverDue_dna = 0 THEN 'Current'
    WHEN DaysOverDue_dna between 1 and 30 THEN '1 - 30'
    WHEN DaysOverDue_dna between 31 and 60 THEN '31 - 60'
    WHEN DaysOverDue_dna between 61 and 90 THEN '61 - 90'
    WHEN DaysOverDue_dna between 91 and 120 THEN '91 - 120'
    WHEN DaysOverDue_dna between 61 and 90 THEN '121 - 150'
    WHEN DaysOverDue_dna between 151 and 180 THEN '151 - 180'
    WHEN DaysOverDue_dna between 181 and 210 THEN '181 - 210'
    WHEN DaysOverDue_dna between 211 and 575 THEN 'WO In di bawah 1 tahun'
    WHEN DaysOverDue_dna between 576 and 1305 THEN 'WO In 1-3 tahun'
    WHEN DaysOverDue_dna between 1306 and 2035 THEN 'WO In 4-5 tahun'
    ELSE 'WO In di atas 5 tahun'
END [DNA Bucket]
FROM ral_dna
),
ral_result_rnk AS(
SELECT
*,
CASE
    WHEN Result in ('Inventory', 'Reposess', 'Pelunasan') THEN 1
    WHEN Result = 'Current' THEN 2
    ELSE 3
END result_rnk
FROM ral_dna_bucket
),
ral_ralno_rnk AS(
SELECT
*,
DENSE_RANK() OVER(PARTITION BY AgreementNo ORDER BY rnk ASC, result_rnk ASC, RALNO DESC) AS final_rnk
FROM ral_result_rnk
)
SELECT
*,
CASE
    WHEN final_rnk = 1 THEN [RESULT GROUP]
    ELSE 'Gagal'
END [RESULT GROUP 3]
FROM ral_ralno_rnk
"""
#cursor.execute(query)

In [5]:
data = pd.read_sql(query, cnxn)

C:\Users\099692\AppData\Local\Temp\ipykernel_20936\3477599559.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, cnxn)


In [ ]:
data.head()

In [ ]:
data.to_excel("RAL.xlsx", index=False)